In [1]:
from PIL import Image, ImageDraw 
from IPython.display import Image as DImage
import sys, os, hashlib

In [5]:
#INTERFACE
pix_mode   = 1 #compare per-pixel
hash_mode  = 1 #compare hashes
black_mode = 0 #compare only black pixels
bin_ratio  = 600 #binarization threshold (0-255*3)
basewidth  = 8 #image width after compression (px)

#init directories
if not os.path.exists(os.getcwd()+'/examples/'):
    os.mkdir('examples')
if not os.path.exists(os.getcwd()+'/binary/'):
    os.mkdir('binary')
if not os.path.exists(os.getcwd()+'/resized/'):
    os.mkdir('resized')
examples_path = os.getcwd()+'/examples/'
binary_path = os.getcwd()+'/binary/'
resized_path = os.getcwd()+'/resized/'

#global variables
names = []
values = []
distances = []
RESULT = ''

In [3]:
input_address = input('Input image address:') #input must be in "" ("test.jpg")

Input image address:"test.jpg"


In [6]:
def address_helper(input_img_addr, mode):
    if mode == 'png':
        input_img_addr = input_img_addr.replace('.jpg','.png')
        input_img_addr = input_img_addr.replace('.gif','.png')
    if mode == 'bin':
        input_img_addr = input_img_addr.replace('/examples/','/binary/')
        input_img_addr = input_img_addr.replace('/resized/','/binary/')
    if mode == 'res':
        input_img_addr = input_img_addr.replace('/examples/','/resized/')
        input_img_addr = input_img_addr.replace('/binary/','/resized/')
    if mode == 'cln':
        input_img_addr = input_img_addr.replace('.jpg','')
        input_img_addr = input_img_addr.replace('.gif','')
        input_img_addr = input_img_addr.replace('.png','')
        input_img_addr = input_img_addr.replace(os.getcwd()+'/binary/','')
        input_img_addr = input_img_addr.replace(os.getcwd()+'/resized/','')
    return input_img_addr

In [7]:
def binarization(input_img_addr):
    input_img = Image.open(input_img_addr) 
    draw = ImageDraw.Draw(input_img)
    input_width = input_img.size[0]
    input_height = input_img.size[1]
    input_pixels = input_img.load()
    
    if ".gif" not in input_img_addr:
        for i in range(input_width):
            for j in range(input_height):
                r = input_pixels[i, j][0]
                g = input_pixels[i, j][1]
                b = input_pixels[i, j][2]
                s = r + g + b
                if (s > bin_ratio):
                    r, g, b = 255, 255, 255
                else:
                    r, g, b = 0, 0, 0
                draw.point((i, j), (r, g, b))
    
    input_img_addr = address_helper(input_img_addr,'png')
    input_img.save(address_helper(input_img_addr,'bin'), "PNG")
    del draw

In [8]:
def gethash(input_img_addr):
    input_img = Image.open(input_img_addr)
    return (hashlib.md5(input_img.tobytes()).hexdigest())

In [9]:
def hamdist(str1, str2, prevMin=None):
    diffs = 0
    if len(str1) != len(str2):
        return max(len(str1),len(str2))
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
            if prevMin is not None and diffs > prevMin:
                return None
    return diffs

In [10]:
def resize(input_img_addr, basewidth):
    img = Image.open(input_img_addr)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth,hsize))
    img.save(address_helper(input_img_addr,'res'), "PNG")

In [11]:
def pix_compare(input_img_addr, example_addr):
    input_img = Image.open(input_img_addr)
    input_pixels = input_img.load()
    example = Image.open(example_addr)
    true = 0 
    
    for i in range(0, input_img.size[0]):
        for j in range(0, input_img.size[1]):
            input_pixel = input_img.getpixel((i, j))
            example_pixel = example.getpixel((i, j))
            if input_pixel == example_pixel:                        
                if not (black_mode == 1 and input_pixels[i, j][0] == 0):
                    true+=1  
    return true

In [12]:
def compare1to1(input_img_addr, example_addr):
    example_addr = address_helper(example_addr,'png')
    input_img_addr = address_helper(input_img_addr,'png')
    
    if pix_mode == 1:
        input_img_addr = os.getcwd()+'/binary/'+input_img_addr         
        values.append(pix_compare(input_img_addr, example_addr))

        
    if hash_mode == 1:
        resize(input_img_addr, basewidth)
        resize(example_addr, basewidth)
        
        example_addr = address_helper(example_addr,'res')
        input_img_addr = address_helper(input_img_addr,'res')
        
        input_hash = gethash(input_img_addr)
        example_hash = gethash(example_addr)
        
        distances.append(hamdist(input_hash, example_hash))
    example_addr = address_helper(example_addr, 'cln')
    names.append(example_addr)   

In [13]:
def compare1toN(input_img_addr):
    #delete previous values
    global names
    global values
    global distances
    names = []
    values = []
    distances = []
    
    examples = os.listdir(examples_path)
    for i in range(len(examples)):
        binarization(examples_path+examples[i])
        compare1to1(input_img_addr,binary_path+examples[i])

In [14]:
def print1toN():
    if pix_mode == 1:
        print('Pixel mode:')
        for i in range(len(names)):
            print(names[i],values[i])
    if hash_mode == 1:
        print('Hash mode:')
        for i in range(len(names)):
            print(names[i],distances[i])

In [15]:
def result1toN():
    if pix_mode == 1:
        print(names[values.index(max(values))])
    if hash_mode == 1:
        print(names[distances.index(min(distances))])

In [17]:
binarization(input_address)
compare1toN(input_address)
print1toN()
result1toN()

Pixel mode:
('test', 47718)
('ctest', 46448)
('compare', 35734)
('test2', 0)
('a', 47718)
Hash mode:
('test', 0)
('ctest', 30)
('compare', 29)
('test2', 30)
('a', 0)
test
test
